# OSSE metrics

In [16]:
from src.utils import rmse_based_scores_from_ds
from src.utils import psd_based_scores_from_ds
test_data = xr.open_dataset('/DATASET/mbeauchamp/xp_osse_rzf2.nc')
mu, sig = rmse_based_scores_from_ds(test_data.sel(time=slice('2012-10-22', '2012-12-02',)), ref_variable='tgt', study_variable='out')
lx , lt = psd_based_scores_from_ds(test_data.sel(time=slice('2012-10-22', '2012-12-02',)), ref_variable='tgt', study_variable='out')
print(mu,sig,lx,lt)

0.94489 0.00992 1.224 9.374
0.9215 0.01694 1.205 13.893


# OSE metrics

In [ ]:
from src.ose.mod_inout import *
from src.ose.mod_interp import *
from src.ose.mod_stats import *
from src.ose.mod_spectral import *
from src.ose.mod_plot import *
from src.ose.utils import *

logger = logging.getLogger()
logger.setLevel(logging.INFO)

# study area
#lon_min+180)%360-180
lon_min = 295.
lon_max = 305.
lat_min = 33.
lat_max = 43.
is_circle = False
time_min = '2017-12-09'
time_max = '2017-12-29'

# Outputs
bin_lat_step = 1.
bin_lon_step = 1.
bin_time_step = '1D'

# Spectral parameter
# C2 parameter
delta_t = 0.9434  # s
velocity = 6.77   # km/s
delta_x = velocity * delta_t
lenght_scale = 1000 # km

# independent along-track
alontrack_independent_dataset= '../src/ose/dt_gulfstream_c2_phy_l3_20161201-20180131_285-315_23-53.nc'
# Read along-track
ds_alongtrack = read_l3_dataset(alontrack_independent_dataset, 
                                           lon_min=lon_min, 
                                           lon_max=lon_max, 
                                           lat_min=lat_min, 
                                           lat_max=lat_max, 
                                           time_min=time_min, 
                                           time_max=time_max)

file_oi = '../src/ose/ssh_alg_h2g_j2g_j2n_j3_s3a_duacs.nc'
file_4dvarnet = '/DATASET/mbeauchamp/xp_ose_rzf2_for_metrics.nc'
test_data = xr.open_dataset('/DATASET/mbeauchamp/xp_ose_rzf2.nc')
test_data = test_data.update({'ssh':(('time','lat','lon'),test_data.out.data)}).to_netcdf(file_4dvarnet)

In [ ]:
# Interpolate maps onto alongtrack dataset for each dataset
res_oi = interp_on_alongtrack(file_oi,
                              ds_alongtrack,
                              lon_min=lon_min,
                              lon_max=lon_max,
                              lat_min=lat_min,
                              lat_max=lat_max,
                              time_min=time_min,
                              time_max=time_max,
                              is_circle=is_circle)
time_alongtrack, lat_alongtrack, lon_alongtrack, ssh_alongtrack, ssh_oi_interp = res_oi                    
ssh_4dvarnet_interp = interp_on_alongtrack(file_4dvarnet,
                              ds_alongtrack,
                              lon_min=lon_min,
                              lon_max=lon_max,
                              lat_min=lat_min,
                              lat_max=lat_max,
                              time_min=time_min,
                              time_max=time_max,
                              is_circle=is_circle)[4]

In [ ]:
# Compute spatial and temporal statistics
leaderboard_nrmse_oi, leaderboard_nrmse_std_oi = compute_stats(time_alongtrack, 
                                                         lat_alongtrack, 
                                                         lon_alongtrack, 
                                                         ssh_alongtrack, 
                                                         ssh_oi_interp, 
                                                         bin_lon_step,
                                                         bin_lat_step, 
                                                         bin_time_step,
                                                         output_filename='spa_stat_OI.nc',
                                                         output_filename_timeseries='TS_OI.nc')           

leaderboard_nrmse_4dvarnet, leaderboard_nrmse_std_4dvarnet = compute_stats(time_alongtrack, 
                                                         lat_alongtrack, 
                                                         lon_alongtrack, 
                                                         ssh_alongtrack, 
                                                         ssh_4dvarnet_interp, 
                                                         bin_lon_step,
                                                         bin_lat_step, 
                                                         bin_time_step,
                                                         output_filename='spa_stat_4DVarNet.nc',
                                                         output_filename_timeseries='TS_4DVarNet.nc')

# Compute spectral scores
compute_spectral_scores(time_alongtrack, 
                        lat_alongtrack, 
                        lon_alongtrack, 
                        ssh_alongtrack, 
                        ssh_oi_interp, 
                        lenght_scale,
                        delta_x,
                        delta_t,
                        'spectrum_oi.nc')   
compute_spectral_scores(time_alongtrack, 
                        lat_alongtrack, 
                        lon_alongtrack, 
                        ssh_alongtrack, 
                        ssh_4dvarnet_interp, 
                        lenght_scale,
                        delta_x,
                        delta_t,
                        'spectrum_4dvarnet.nc') 

In [ ]:
leaderboard_psds_score_4dvarnet = -999
leaderboard_psds_score_oi = plot_psd_score('spectrum_oi.nc')  
leaderboard_psds_score_4dvarnet = plot_psd_score('spectrum_4dvarnet.nc')  

In [ ]:
# Print leaderboard
data = np.array([['OI','4DVarNet'],
                 [leaderboard_nrmse_oi,
                  leaderboard_nrmse_4dvarnet],
                 [leaderboard_nrmse_std_oi,
                 leaderboard_nrmse_std_4dvarnet],
                 [int(leaderboard_psds_score_oi),int(leaderboard_psds_score_4dvarnet)]]).T
                                                 
Leaderboard = pd.DataFrame(data, 
                           columns=['Method', 
                                    "µ(RMSE) ", 
                                    "σ(RMSE)", 
                                    'λx (km)'])
print("Summary of the leaderboard metrics:")
Leaderboard
print(Leaderboard.to_markdown()) 
os.remove('spa_stat_OI.nc')
os.remove('spa_stat_4DVarNet.nc')
os.remove('TS_OI.nc')
os.remove('TS_4DVarNet.nc')
os.remove('spectrum_oi.nc')
os.remove('spectrum_4dvarnet.nc')
os.remove(file_4dvarnet)